In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 85.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
import numpy as np

In [4]:
!tree /content/drive/MyDrive


/bin/bash: line 1: tree: command not found


In [5]:
from pathlib import Path
img_path = Path('/content/drive/MyDrive/mlp_rawdata/tif_img')

img_names = [f.name for f in img_path.iterdir() if f.is_file()]


In [6]:
gpkg_path = '/content/drive/MyDrive/mlp_rawdata/TN_MAPINDX_5K.gpkg'

gpkg = gpd.read_file(gpkg_path)

gpkg = gpkg.to_crs(epsg=5186)


In [7]:
gpkg[gpkg['MAPIDCD_NO']=='37608078'].geometry.total_bounds

array([193372.51997588, 550054.67002869, 195583.15009928, 552830.8499952 ])

In [8]:
shp_path = '/content/drive/MyDrive/mlp_rawdata/AL_D010_11_20240404/AL_D010_11_20240404.shp'
shp = gpd.read_file(shp_path, encoding='cp949')
shp.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A20,A21,A22,A23,A24,A25,A26,A27,A28,geometry
0,32,1991201839054527769900000000,1111017500107040000,1111017500,서울특별시 종로구 숭인동,704,1,일반,01000,단독주택,...,N,B00100000000T30Z9,2024-04-01,11110,None,None,3.0,0.0,2017-05-30,"POLYGON ((201913.638 553085.323, 201912.228 55..."
1,34,1967201343844527773800000000,1111017500100560024,1111017500,서울특별시 종로구 숭인동,56-24,1,일반,01000,단독주택,...,N,B00100000000T311B,2024-04-01,11110,None,None,3.0,0.0,2018-06-19,"POLYGON ((201413.758 553078.463, 201410.198 55..."
2,36,1962201375194527742000000000,1111017500100560054,1111017500,서울특별시 종로구 숭인동,56-54,1,일반,01000,단독주택,...,N,B00100000000T313D,2024-04-01,11110,None,None,2.0,0.0,2017-02-24,"POLYGON ((201442.598 553075.703, 201442.298 55..."
3,37,1979201388484527710100000000,1111017500100560053,1111017500,서울특별시 종로구 숭인동,56-53,1,일반,01000,단독주택,...,N,B00100000000T314E,2024-04-01,11110,None,None,2.0,1.0,2018-11-06,"POLYGON ((201458.568 553069.403, 201453.548 55..."
4,38,0000200468294527764000000000,1111016500100280011,1111016500,서울특별시 종로구 이화동,28-11,1,일반,None,None,...,None,B00100000000T315F,2024-04-01,11110,None,None,0.0,0.0,None,"POLYGON ((200540.538 553080.543, 200536.438 55..."


In [10]:
for img in img_names:

  map_number = img.split("_")[2].split(".")[0]
  map = gpkg[gpkg['MAPIDCD_NO'] == map_number]

  bounds = map.total_bounds
  pixel = 0.25
  print(bounds)

  width = int((bounds[2] - bounds[0])/ pixel)
  height = int((bounds[3] - bounds[1]) / pixel)

  transform = rasterio.transform.from_origin(bounds[0], bounds[3], pixel, pixel)

  shapes = ((geom, i+1) for i, geom in enumerate(shp.geometry))
  raterized = features.rasterize(
      shapes = shapes,
      out_shape = (height, width),
      transform = transform
  )
  np_path = '/content/drive/MyDrive/mlp_rawdata/label_np/' + map_number + '.npy'

  np.save(np_path, raterized)
  print(np_path, ': 저장', width, height)





[193372.51997588 550054.67002869 195583.15009928 552830.8499952 ]
/content/drive/MyDrive/mlp_rawdata/label_np/37608078.npy : 저장 8842 11104


In [11]:
print(height, width)

11104 8842


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import rasterio
import numpy as np
from pathlib import Path
from transformers import Mask2FormerForUniversalSegmentation
import torchvision.transforms.functional as F
from torchvision import transforms


# 1. 데이터셋 클래스 정의
#    - 이미지와 라벨을 함께 리사이즈하고 정규화하는 기능 추가

class BuildingDataset(Dataset):
    def __init__(self, img_dir, label_dir, resize_shape=(512, 512)):
        self.img_path = Path(img_dir)
        self.label_path = Path(label_dir)
        self.img_files = sorted([f for f in self.img_path.iterdir() if f.suffix == '.tif'])
        self.resize_shape = resize_shape # 리사이즈할 크기 (높이, 너비)

        # 이미지 정규화를 위한 평균(mean)과 표준편차(std)
        # ImageNet 데이터셋에서 널리 사용되는 값
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        # (A) 이미지 파일(.tif) 읽기
        img_filepath = self.img_files[idx]
        with rasterio.open(img_filepath) as src:
            image = src.read().astype(np.float32)

        # (B) 라벨 파일(.npy) 읽기
        try:
            map_number = img_filepath.name.split("_")[2].split(".")[0]
            label_filename = map_number + '.npy'
            label_filepath = self.label_path / label_filename
            label = np.load(label_filepath).astype(np.int64)
        except (IndexError, FileNotFoundError):
            print(f"경고: {img_filepath.name}에 대한 라벨 파일을 찾을 수 없거나 형식이 맞지 않습니다.")
            label = np.zeros((image.shape[1], image.shape[2]), dtype=np.int64) # (높이, 너비) 순서

        # NumPy 배열을 PyTorch 텐서로 변환
        image_tensor = torch.from_numpy(image)
        # 라벨은 채널 차원이 없으므로 [높이, 너비] 형태의 2D 텐서로 만듭니다.
        # unsqueeze(0)를 통해 [1, 높이, 너비] 형태로 변경하여 리사이즈 준비
        label_tensor = torch.from_numpy(label).unsqueeze(0)

        ## 리사이즈 및 정규화 부분##
        # 이미지 리사이즈: 보간법(interpolation)은 BILINEAR 사용
        image_tensor = F.resize(image_tensor, self.resize_shape, interpolation=F.InterpolationMode.BILINEAR)

        # 라벨 리사이즈: 라벨 값(건물 ID)이 변하면 안 되므로, 가장 가까운 값으로 채우는 NEAREST 사용
        label_tensor = F.resize(label_tensor, self.resize_shape, interpolation=F.InterpolationMode.NEAREST)

        # 이미지 정규화
        image_tensor = self.normalize(image_tensor)

        # 라벨 텐서의 불필요한 채널 차원 제거: [1, 높이, 너비] -> [높이, 너비]
        label_tensor = label_tensor.squeeze(0)


        return image_tensor, label_tensor


# 2. 데이터셋 및 데이터로더 생성 (동일)
IMG_DIRECTORY = '/content/drive/MyDrive/mlp_rawdata/tif_img'
LABEL_DIRECTORY = '/content/drive/MyDrive/mlp_rawdata/label_np'

print("데이터셋을 생성합니다...")
# 데이터셋 생성 시 리사이즈 크기를 지정할 수 있습니다.
building_dataset = BuildingDataset(img_dir=IMG_DIRECTORY, label_dir=LABEL_DIRECTORY, resize_shape=(512, 512))
data_loader = DataLoader(building_dataset, batch_size=1, shuffle=True)
print(f"데이터셋에 총 {len(building_dataset)}개의 샘플이 있습니다.")


# 3. Mask2Former 모델 및 Swin Transformer 백본 로드 (동일)
print("\n사전 학습된 Mask2Former 모델과 Swin 백본을 로드합니다...")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-base-coco-instance")
model.eval()

swin_backbone = model.model.pixel_level_module.encoder
print(f"성공적으로 {type(swin_backbone).__name__} 백본을 추출했습니다.")


# 4. 데이터셋의 첫 번째 배치로 백본 실행 테스트 (동일)
try:
    image_batch, label_batch = next(iter(data_loader))
    print(f"\n데이터 로더에서 첫 번째 이미지 배치(형태: {image_batch.shape})를 가져왔습니다.") # 리사이즈된 크기 확인

    with torch.no_grad():
        feature_maps = swin_backbone(image_batch, output_hidden_states=True, return_dict=False)

    hidden_states = feature_maps[1]
    print("\nSwin Transformer 백본이 추출한 다중 스케일 특징 맵:")
    for i, feature_map in enumerate(hidden_states):
        print(f"  - Stage {i} 특징 맵 형태: {feature_map.shape}")

except StopIteration:
    print("\n오류: 데이터셋이 비어있거나 데이터 로더에서 데이터를 가져올 수 없습니다.")



데이터셋을 생성합니다...
데이터셋에 총 1개의 샘플이 있습니다.

사전 학습된 Mask2Former 모델과 Swin 백본을 로드합니다...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


성공적으로 SwinBackbone 백본을 추출했습니다.


/usr/local/lib/python3.12/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



데이터 로더에서 첫 번째 이미지 배치(형태: torch.Size([1, 3, 512, 512]))를 가져왔습니다.

Swin Transformer 백본이 추출한 다중 스케일 특징 맵:
  - Stage 0 특징 맵 형태: torch.Size([1, 16384, 128])
  - Stage 1 특징 맵 형태: torch.Size([1, 16384, 128])
  - Stage 2 특징 맵 형태: torch.Size([1, 4096, 256])
  - Stage 3 특징 맵 형태: torch.Size([1, 1024, 512])
  - Stage 4 특징 맵 형태: torch.Size([1, 256, 1024])


##### <백본 파트 역할>

우리 모델 Mask2Former에서 백본의 역할은, 입력된 항공사진(.tif)으로부터 다양한 수준의 특징 정보(Feature Maps)를 추출하여, 후속 모듈인 픽셀 디코더(Pixel Decoder)와 트랜스포머 디코더(Transformer Decoder)로 전달하는 것입니다.​



##### <최종 구현 코드 및 결과>

아래 코드는 우리 프로젝트의 데이터를 사용하여 백본(Swin Transformer)을 실행하고, 그 결과물을 확인하는 최종 코드입니다.







##### \[핵심 코드 로직]

1. ###### 데이터셋 정의 (BuildingDataset 클래스)

* 항공사진(.tif)과 우리가 전처리하여 생성한 건물 라벨(.npy)을 한 쌍으로 묶어 불러옵니다
* 메모리 문제를 해결하고 모델 학습 효율을 높이기 위해, 거대한 원본 이미지를 일정한 크기(512x512)로 리사이즈하고 정규화하는 전처리 과정을 포함했습니다.

###### 2\. 모델 로드 및 백본 추출

* Hugging Face의 사전 학습된 Mask2FormerForUniversalSegmentation 모델을 불러옵니다.
* 모델 내부에서 실제 특징 추출을 담당하는 SwinBackbone 부분만 정확히 추출하여 사용합니다.

###### 3\. 백본 실행 및 특징 맵 추출

전처리된 이미지 데이터를 백본에 입력하여, 최종 결과물인 다중 스케일 특징 맵(Multi-scale Feature Maps)을 추출했습니다.







##### \[실행 결과]

데이터 로더에서 첫 번째 이미지 배치(형태: torch.Size(\[1, 3, 512, 512]))를 가져왔습니다.

Swin Transformer 백본이 추출한 다중 스케일 특징 맵:

&nbsp; - Stage 0 특징 맵 형태: torch.Size(\[1, 16384, 128])

&nbsp; - Stage 1 특징 맵 형태: torch.Size(\[1, 16384, 128])

&nbsp; - Stage 2 특징 맵 형태: torch.Size(\[1, 4096, 256])

&nbsp; - Stage 3 특징 맵 형태: torch.Size(\[1, 1024, 512])

&nbsp; - Stage 4 특징 맵 형태: torch.Size(\[1, 256, 1024])



-> 결과물의 의미: 픽셀 디코더와 트랜스포머 디코더로 전달될 데이터

위 결과에서 Swin Transformer 백본이 추출한 다중 스케일 특징 맵이 backbone 파트의 최종 결과물이며, 다른 파트로 전달될 입력 데이터입니다.



* 다중 스케일(Multi-scale)의 의미: 이 특징 맵들은 하나의 이미지를 다양한 관점에서 분석한 결과물입니다.​

-초기 Stage (고해상도): 이미지의 세밀한 경계선, 모서리 등 '디테일' 정보를 담고 있습니다.

-후기 Stage (저해상도): 이미지의 전체적인 구조, 객체의 형태 등 '맥락' 정보를 담고 있습니다.

* 팀원 파트와의 연관성:

-픽셀 디코더 파트:  다중 스케일 특징 맵들(hidden\_states)을 입력으로 받아, 이를 다시 융합하고 업샘플링하여 최종 마스크를 예측하는 데 필요한 고해상도의 픽셀별 임베딩(per-pixel embeddings)을 생성하게 됩니다.​

-트랜스포머 디코더 파트:  이 특징 맵들과 학습 가능한 쿼리(queries)를 함께 사용하여, 이미지에 어떤 객체들이 있는지, 그리고 각 객체가 어떤 픽셀들을 포함하는지에 대한 정보를 추출하게 됩니다.​





##### <결론 및 다음 단계>

여기서 구현한 코드의 hidden\_states 변수가 다른 파트의 모듈로 전달될 입력 데이터이니, 이를 기반으로 파트 연동을 준비해 주시면 됩니다.

